# Implementing Hybrid Search using Pinecone, Langchain

In [34]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings

load_dotenv(dotenv_path="../.env")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [35]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


index_name = "hybrid-search"

pc.create_index(
    name=index_name,
    dimension= 1536,
    metric="dotproduct", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [36]:
index = pc.Index(index_name)
embeddings = OpenAIEmbeddings()

In [37]:
from pinecone_text.sparse import BM25Encoder

bm25 = BM25Encoder().default()

In [38]:
corpus = ["The quick brown fox jumps over the lazy dog",
          "The lazy dog is brown",
          "The fox is brown"]

# Initialize BM25 and fit the corpus.
bm25.fit(corpus)

bm25.dump("bm25_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [39]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
retriever = PineconeHybridSearchRetriever(embeddings= embeddings, sparse_encoder = bm25, index = index)

In [40]:
retriever.add_texts(
    [
        """Malcolm X was one of the most dynamic, dramatic and influential figures of the civil rights era. He was an apostle of black nationalism, self respect, and uncompromising resistance to white oppression. Malcolm X was a polarizing figure who both energized and divided African Americans, while frightening and alienating many whites. He was an unrelenting truth-teller who declared that the mainstream civil rights movement was naïve in hoping to secure freedom through integration and nonviolence. The blazing heat of Malcolm X's rhetoric sometimes overshadowed the complexity of his message, especially for those who found him threatening in the first place. Malcolm X was assassinated at age 39, but his political and cultural influence grew far greater in the years after his death than when he was alive.

Malcolm X is now popularly seen as one of the two great martyrs of the 20th century black freedom struggle, the other being his ostensible rival, the Rev. Martin Luther King Jr. But in the spring of 1964, when Malcolm X gave his "Ballot or the Bullet" speech, he was regarded by a majority of white Americans as a menacing character. Malcolm X never directly called for violent revolution, but he warned that African Americans would use "any means necessary" – especially armed self defense – once they realized just how pervasive and hopelessly entrenched white racism had become.1

He was born Malcolm Little in 1925 in Omaha, Nebraska. His father, Earl, was a Baptist preacher and follower of the black nationalist Marcus Garvey. Earl Little's political activism provoked threats from the Ku Klux Klan. After the family moved to Lansing, Michigan, white terrorists burned the Littles' home. A defiant Earl Little shot at the arsonists as they got away. In 1931, Malcolm's father was found dead. His family suspected he'd been murdered by white vigilantes. Malcolm's mother, Louise, battled mental illness and struggled to care for her eight children during the Great Depression. She was committed to a state mental institution when Malcolm was 12. He and the other young children were scattered among foster families. After completing the eighth grade, Malcolm Little dropped out when a teacher told him that his dream of becoming a lawyer was unrealistic for a "nigger."2

As a teenager, Malcolm Little made his way to New York, where he took the street name Detroit Red and became a pimp and petty criminal. In 1946, Malcolm Little was sent to prison for burglary. He read voraciously while serving time and converted to the Black Muslim faith. He joined the Nation of Islam (NOI) and changed his name to Malcolm X, eliminating that part of his identity he called a white-imposed slave name.

Malcom X was released in 1952 after six years in prison. With his charisma and eloquence, Malcolm rose rapidly in the Nation of Islam. He became the chief spokesman and field recruiter for NOI leader Elijah Muhammad. As historian Peniel Joseph describes it, NOI's unorthodox interpretation of Islam was mixed with a doctrine of black personal responsibility and economic self-sufficiency, along with "theological fundamentalism, anti-white mythology, and total racial separation as the means to black redemption."3 Wearing impeccable suits, maintaining an air of fierce dignity and adhering to a strict code of moral propriety, Malcolm X was a living demonstration of how the NOI could save a wayward people from racial submission and personal self-destruction. The Nation dismissed the conventional civil rights movement – with its protest marches and demands for equal rights legislation -- as impotent and misguided. As Malcolm X declared in this speech, the only effective solution to racial inequality was black economic and social separatism."""
    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
retriever.invoke("Malcolm X")

[Document(page_content='Malcolm X was one of the most dynamic, dramatic and influential figures of the civil rights era. He was an apostle of black nationalism, self respect, and uncompromising resistance to white oppression. Malcolm X was a polarizing figure who both energized and divided African Americans, while frightening and alienating many whites. He was an unrelenting truth-teller who declared that the mainstream civil rights movement was naïve in hoping to secure freedom through integration and nonviolence. The blazing heat of Malcolm X\'s rhetoric sometimes overshadowed the complexity of his message, especially for those who found him threatening in the first place. Malcolm X was assassinated at age 39, but his political and cultural influence grew far greater in the years after his death than when he was alive.\n\nMalcolm X is now popularly seen as one of the two great martyrs of the 20th century black freedom struggle, the other being his ostensible rival, the Rev. Martin Lu